# Spinor connection, Dirac equation and Wald solution

## Basic initialization of the lorentzian manifold

Initialization of the Kerr manifold $\mathcal M_{Kerr}$ atlas with Boyer-Lindquist chart (BL) and of the metric tensor $g$ :

In [1]:
%display latex

M = Manifold(4, 'M', latex_name=r'\mathcal{M}', structure='Lorentzian')
#BL.<t,r,th,ph> = M.chart(r"t r th:(0,pi):\theta ph:(0,2*pi):\phi") #boyer-lindquist chart definition
SD.<t, r, th, ph> = M.chart(r"t r:(0,+oo) th:(0,pi):\theta ph:(0,2*pi):\phi:periodic")

var('m', domain='real')
#var('m, a', domain='real') #definition of Kerr metric parameters

g = M.metric() #we put the label g on the metric tensor

#rho2 = r^2 + (a*cos(th))^2
#Delta = r^2 -2*m*r + a^2

#g[0,0] = -(1-2*m*r/rho2)
#g[0,3] = -2*a*m*r*sin(th)^2/rho2
#g[1,1], g[2,2] = rho2/Delta, rho2
#g[3,3] = (r^2+a^2+2*m*r*(a*sin(th))^2/rho2)*sin(th)^2
g[0, 0] = - (1 - 2*m/r)
g[1, 1] = 1/(1 - 2*m/r)
g[2, 2] = r^2
g[3, 3] = r^2*sin(th)^2

g.display()

g = (2*m/r - 1) dt*dt - 1/(2*m/r - 1) dr*dr + r^2 dth*dth + r^2*sin(th)^2 dph*dph

## Initialization of the orthonormal tetrad

Definition of the orthonormal tetrad frame $e_{(a)}$ (Chandrasekar notation) and its dual frame $e^{(a)} = \eta^{(a)(b)}g(e_{(b)},-)$ :

In [2]:
e0 = M.vector_field(1/sqrt(1-2*m/r), 0, 0, 0, frame=SD.frame(), name='e_{(0)}')
e1 = M.vector_field(0, sqrt(1-2*m/r), 0, 0, frame=SD.frame(), name='e_{(1)}')
e2 = M.vector_field(0, 0, 1/r, 0, frame=SD.frame(), name='e_{(2)}')
e3 = M.vector_field(0, 0, 0, 1/(r*sin(th)), frame=SD.frame(), name='e_{(3)}')


e = M.vector_frame('e', (e0, e1, e2, e3)) #costruction of the tetrad collection object e_{a}
de = e.coframe() #construction of the tetrad coframe object e^{a}  

## ST bundle

In [3]:
from ST_bundle.ST_bundle import ST_bundle

In [4]:
e_sfb = ST_bundle(M,e)

In [16]:
psi0 = function("psi0", latex_name = "\\psi_{0}", nargs = 4)
psi1 = function("psi1" ,latex_name = "\\psi_{1}", nargs = 4)
psi2 = function("psi2" ,latex_name = "\\psi_{2}", nargs = 4)
psi3 = function("psi3" ,latex_name = "\\psi_{3}", nargs = 4)


psi0_ = M.scalar_field({SD: psi0(r,t,th,ph)})
psi1_ = M.scalar_field({SD: psi1(r,t,th,ph)})
psi2_ = M.scalar_field({SD: psi2(r,t,th,ph)})
psi3_ = M.scalar_field({SD: psi3(r,t,th,ph)})

psi_sb = e_sfb.sbundle.section({e_sfb.sframe: [psi0_,psi1_,psi2_,psi3_]}, "\\psi")
psi = e_sfb.spin_tensor([],(1,0),psi_sb)
Gamma = e_sfb.Dirac_matrices("up")

nabla_s = e_sfb.sconnection()
nabla_t = e_sfb.tconnection()
nabla_tot = e_sfb.totconnection()

In [6]:
nabla_tot

Total connection on the ST bundle

In [7]:
nabla_tot.display('t')

connection (0,1) of bundle connection \nabla_t w.r.t. Local frame (F|_M, (e_0,e_1,e_2,e_3)) = m/r^2 dt 
connection (1,0) of bundle connection \nabla_t w.r.t. Local frame (F|_M, (e_0,e_1,e_2,e_3)) = m/r^2 dt 
connection (1,2) of bundle connection \nabla_t w.r.t. Local frame (F|_M, (e_0,e_1,e_2,e_3)) = sqrt(-2*m + r)/sqrt(r) dth 
connection (1,3) of bundle connection \nabla_t w.r.t. Local frame (F|_M, (e_0,e_1,e_2,e_3)) = sqrt(-2*m + r)*sin(th)/sqrt(r) dph 
connection (2,1) of bundle connection \nabla_t w.r.t. Local frame (F|_M, (e_0,e_1,e_2,e_3)) = -sqrt(-2*m + r)/sqrt(r) dth 
connection (2,3) of bundle connection \nabla_t w.r.t. Local frame (F|_M, (e_0,e_1,e_2,e_3)) = cos(th) dph 
connection (3,1) of bundle connection \nabla_t w.r.t. Local frame (F|_M, (e_0,e_1,e_2,e_3)) = -sqrt(-2*m + r)*sin(th)/sqrt(r) dph 
connection (3,2) of bundle connection \nabla_t w.r.t. Local frame (F|_M, (e_0,e_1,e_2,e_3)) = -cos(th) dph

In [8]:
curv_tot = nabla_tot(e,nabla_s(e,psi_sb))

In [9]:
curv_tot = curv_tot.swap_tindices()-curv_tot

In [10]:
Riemann = M.metric().riemann()

In [11]:
Riemann = Riemann.down(g)

In [12]:
Riem = e_sfb.set_scalar_from_Components(["down","down","down","down"],Riemann.comp(e))

In [13]:
curv_from_riem = (1/4)*Riem.contract([0,1],Gamma@Gamma,[0,1])

In [14]:
curv_from_riem_psi = curv_from_riem@psi_sb

In [15]:
curv_tot == curv_from_riem_psi

True

In [16]:
curv_tot_2form = curv_from_riem.get_tangent_tensor()

In [17]:
for i in range(0,4):
    for j in range(0,4):
        if(curv_tot_2form[i][j] != nabla_s.curvature_form(j,i)): print("error")